In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances

In [2]:
data = pd.read_csv("/Users/charlottefelius/documents/wids2022/WIDS/data/train.csv")
sub = pd.read_csv("/Users/charlottefelius/documents/wids2022/WIDS/data/test.csv")

In [3]:
data.head(2)
sub.head(2)

,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,...,days_below_0F,days_above_80F,days_above_90F,days_above_100F,days_above_110F,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,id
0,7,State_1,Commercial,Grocery_store_or_food_market,28484.0,1994.0,37.0,2.4,38,50.596774,...,0,29,5,2,0,NaN,NaN,NaN,NaN,75757
1,7,State_1,Commercial,Grocery_store_or_food_market,21906.0,1961.0,55.0,45.7,38,50.596774,...,0,29,5,2,0,NaN,NaN,NaN,NaN,75758


In [4]:
test = data[data["State_Factor"] == "State_11"]

In [5]:
len(test)

6412

In [6]:
print("Amount of rows and cols respectively (testset)")
print(str(len(sub)) + ", " + str(len(sub.columns)) + "\n")
print("Column names:")
print([i.strip("\n") for i in sub.columns])
print()
# print("Missing data:")
# print(data.describe())

Amount of rows and cols respectively (testset)
9705, 63

Column names:
['Year_Factor', 'State_Factor', 'building_class', 'facility_type', 'floor_area', 'year_built', 'energy_star_rating', 'ELEVATION', 'january_min_temp', 'january_avg_temp', 'january_max_temp', 'february_min_temp', 'february_avg_temp', 'february_max_temp', 'march_min_temp', 'march_avg_temp', 'march_max_temp', 'april_min_temp', 'april_avg_temp', 'april_max_temp', 'may_min_temp', 'may_avg_temp', 'may_max_temp', 'june_min_temp', 'june_avg_temp', 'june_max_temp', 'july_min_temp', 'july_avg_temp', 'july_max_temp', 'august_min_temp', 'august_avg_temp', 'august_max_temp', 'september_min_temp', 'september_avg_temp', 'september_max_temp', 'october_min_temp', 'october_avg_temp', 'october_max_temp', 'november_min_temp', 'november_avg_temp', 'november_max_temp', 'december_min_temp', 'december_avg_temp', 'december_max_temp', 'cooling_degree_days', 'heating_degree_days', 'precipitation_inches', 'snowfall_inches', 'snowdepth_inches', 

In [7]:
print("Amount of rows and cols respectively")
print(str(len(data)) + ", " + str(len(data.columns)) + "\n")
print("Column names:")
print([i.strip("\n") for i in data.columns])
print()
# print("Missing data:")
# print(data.describe())

Amount of rows and cols respectively
75757, 64

Column names:
['Year_Factor', 'State_Factor', 'building_class', 'facility_type', 'floor_area', 'year_built', 'energy_star_rating', 'ELEVATION', 'january_min_temp', 'january_avg_temp', 'january_max_temp', 'february_min_temp', 'february_avg_temp', 'february_max_temp', 'march_min_temp', 'march_avg_temp', 'march_max_temp', 'april_min_temp', 'april_avg_temp', 'april_max_temp', 'may_min_temp', 'may_avg_temp', 'may_max_temp', 'june_min_temp', 'june_avg_temp', 'june_max_temp', 'july_min_temp', 'july_avg_temp', 'july_max_temp', 'august_min_temp', 'august_avg_temp', 'august_max_temp', 'september_min_temp', 'september_avg_temp', 'september_max_temp', 'october_min_temp', 'october_avg_temp', 'october_max_temp', 'november_min_temp', 'november_avg_temp', 'november_max_temp', 'december_min_temp', 'december_avg_temp', 'december_max_temp', 'cooling_degree_days', 'heating_degree_days', 'precipitation_inches', 'snowfall_inches', 'snowdepth_inches', 'avg_temp

#### Missing data entries

In [8]:
def missing_data(dataframe):
    
    """
    
    Method for calculating missing data
    input: dataframe
    output: sorted list (descending) with colname, #missing values and percentage of total values
    
    """
    
    length = len(dataframe)
    missing = []
    
    for column in dataframe.columns:
        difference = length - dataframe[column].count()
        
        if difference > 0:
            missing.append((column, difference, str('{0:.2f}'.format(difference/length*100)) +"%"))
    
    missing.sort(key = lambda x: x[1], reverse=True)
    
    return missing
        
result = missing_data(data)
result2 = missing_data(sub)

for i in result:
    print(i)

print()
print("Missing values testset:")

for i in result2:
    print(i)

('days_with_fog', 45796, '60.45%')
('direction_peak_wind_speed', 41811, '55.19%')
('direction_max_wind_speed', 41082, '54.23%')
('max_wind_speed', 41082, '54.23%')
('energy_star_rating', 26709, '35.26%')
('year_built', 1837, '2.42%')

Missing values testset:
('days_with_fog', 9117, '93.94%')
('direction_max_wind_speed', 8575, '88.36%')
('direction_peak_wind_speed', 8575, '88.36%')
('max_wind_speed', 8575, '88.36%')
('energy_star_rating', 2254, '23.23%')
('year_built', 92, '0.95%')


In [9]:
# year built probably missing if built a long time ago, hence impute with some things.
# delete everything with higher then 50 percent missing stuff

data_res = data[data['building_class'] == "Residential"]
data_com = data[data['building_class'] == "Commercial"]
sub_res = sub[sub['building_class'] == "Residential"]
sub_com = sub[sub['building_class'] == "Commercial"]

In [10]:
result_ = missing_data(data_res)
result_1 = missing_data(data_com)
result_2 = missing_data(sub_res)
result__2 = missing_data(sub_com)

reslist = [result_, result_1, result_2, result__2]

for i in reslist:
    print(i)
    print()

[('days_with_fog', 24488, '56.22%'), ('direction_peak_wind_speed', 20889, '47.96%'), ('direction_max_wind_speed', 20403, '46.84%'), ('max_wind_speed', 20403, '46.84%'), ('energy_star_rating', 17602, '40.41%'), ('year_built', 408, '0.94%')]

[('days_with_fog', 21308, '66.18%'), ('direction_peak_wind_speed', 20922, '64.98%'), ('direction_max_wind_speed', 20679, '64.22%'), ('max_wind_speed', 20679, '64.22%'), ('energy_star_rating', 9107, '28.28%'), ('year_built', 1429, '4.44%')]

[('days_with_fog', 3882, '94.73%'), ('direction_max_wind_speed', 3656, '89.21%'), ('direction_peak_wind_speed', 3656, '89.21%'), ('max_wind_speed', 3656, '89.21%'), ('energy_star_rating', 749, '18.28%'), ('year_built', 10, '0.24%')]

[('days_with_fog', 5235, '93.37%'), ('direction_max_wind_speed', 4919, '87.73%'), ('direction_peak_wind_speed', 4919, '87.73%'), ('max_wind_speed', 4919, '87.73%'), ('energy_star_rating', 1505, '26.84%'), ('year_built', 82, '1.46%')]



In [11]:
#### Data types
data.dtypes

Year_Factor                    int64
State_Factor                  object
building_class                object
facility_type                 object
floor_area                   float64
                              ...   
direction_peak_wind_speed    float64
max_wind_speed               float64
days_with_fog                float64
site_eui                     float64
id                             int64
Length: 64, dtype: object

In [12]:
# State_Factor object
# building_class object
# facility_type object
objecttypes = ["State_Factor", "building_class", "facility_type"]

for col in objecttypes:
    print(col)
    print(data[col].value_counts())
    print()
    
for col in objecttypes:
    print(col)
    print(sub[col].value_counts())
    print()

State_Factor
State_6     50840
State_11     6412
State_1      5618
State_2      4871
State_4      4300
State_8      3701
State_10       15
Name: State_Factor, dtype: int64

building_class
Residential    43558
Commercial     32199
Name: building_class, dtype: int64

facility_type
Multifamily_Uncategorized                    39455
Office_Uncategorized                         12512
Education_Other_classroom                     3860
Lodging_Hotel                                 2098
2to4_Unit_Building                            1893
Commercial_Other                              1744
5plus_Unit_Building                           1273
Warehouse_Nonrefrigerated                     1255
Retail_Uncategorized                          1130
Education_College_or_university               1056
Nursing_Home                                   772
Education_Uncategorized                        709
Mixed_Use_Commercial_and_Residential           672
Lodging_Dormitory_or_fraternity_sorority       669
Wareho

In [13]:
alltemps = [string for string in data.columns if 'temp' in string]
allstates = list(data["State_Factor"].unique())
del allstates[3]

In [14]:
allstates
ztop

NameError: name 'ztop' is not defined

In [15]:
def calc_vec(state, dataset):
    vec = []
    
    temp = dataset[dataset["State_Factor"] == state]
    
    for t in alltemps:
            vec.append(temp[t].mean())
    
    return np.asarray(vec)

def euclidian_distance_extensive(original, vectors, dataset):
    
    minval = 999
    name = "Null"
    
    for s in allstates:
        temp = dataset[dataset["State_Factor"] == s]
        
        orig = []
        diff = []
        
        for t in alltemps:
            orig.append(original[t].mean())
            diff.append(temp[t].mean())

        o = np.asarray(orig)
        d = np.asarray(diff)

        dist = np.linalg.norm(o-d)
        cosinesim = sklearn.metrics.pairwise.cosine_similarity(o, d)
        print(f'{s} has value {dist} and cosine sim {cosinesim}')
        
        if dist < minval:
            minval = dist
            name = s
    
    print()
    print(f"Final value: ({name, minval})")
    return (name, minval)

In [16]:
state_values = dict()

for i in allstates:
    state_values[i] = calc_vec(i, data)

In [17]:
# filter state 6 and replace by most resembling state in terms of temperature.
data2 = data.copy()

In [18]:
newdf = pd.DataFrame(state_values)

In [19]:
newdf.corr()

,State_1,State_2,State_4,State_8,State_10,State_11
State_1,1.000000,0.946620,0.947941,0.955325,0.942659,0.972382
State_2,0.946620,1.000000,0.991296,0.996553,0.998194,0.954647
State_4,0.947941,0.991296,1.000000,0.989970,0.992244,0.959317
State_8,0.955325,0.996553,0.989970,1.000000,0.997024,0.961671
State_10,0.942659,0.998194,0.992244,0.997024,1.000000,0.953749
State_11,0.972382,0.954647,0.959317,0.961671,0.953749,1.000000


In [20]:
state6 = data2[data2["State_Factor"] == "State_6"]
state62 = data2[data2["State_Factor"] == "State_6"][:1000]

In [35]:
# Filter data 2

def get_vector(row):
    
    vec = []
    for col in alltemps:
        vec.append(row[col])
    
    return np.asarray(vec)

def get_distance(row, distances):
    
    max_dist = 0
    state = "Null"
    
    vec = get_vector(row)

    for key, value in distances.items():
        
#         dist = np.linalg.norm(value-vec)
        dist = cosine_similarity(vec.reshape(1,-1),value.reshape(1,-1))
#         print(f"LOG: DIST (COS) ={dist}, STATE={key}")
        
        if dist > max_dist:
            max_dist = dist
            state = key
        
#     print(f"LOG: MIN_SIM={max_dist}, STATE={state}")
    return max_dist, state

# test

In [36]:
ids = []
keys = []

for i in range(len(state6)):
    row = state6.iloc[i]
    ids.append(row['id'])
    dist, k = get_distance(row, state_values)
    keys.append(k)

In [37]:
from collections import Counter

c = Counter(keys)

c

Counter({'State_10': 50832, 'State_8': 5, 'State_2': 1, 'State_4': 2})

In [ ]:
stop

In [ ]:
ids[:10]

In [ ]:
set(keys)

In [ ]:
euclidian_distance_extensive(state6, allstates, data)

In [ ]:
alltemps = [string for string in state6.columns if 'temp' in string]

In [ ]:
state11 = data[data["State_Factor"] == "State_2"]

In [ ]:
for t in alltemps:
    print(t, state11[t].mean())

In [ ]:
state11.describe()

#### Distribution of values

In [ ]:
def distribution_plots(dataframe):
    
    """
    
    input: dataframe
    output: plots with distribution of each column
    
    """
    
    for column in dataframe.columns:
        plt.hist(dataframe[column], bins = 35)
        plt.title(column)
        plt.show()

In [ ]:
distribution_plots(sub)

#### Heatmap (correlations of columns)

In [ ]:
sns.heatmap(data.corr())

In [ ]:
for i, j in data.corr()["year_built"].iteritems():
    if abs(j) > 0.1:
        print(i, j)

In [ ]:
def check_correlation(dataframe,colname, min_corr = 0.1):
    
    """
    
    Checks the correlations from a column with all other columns
    input: column name -> string, min_corr -> float
    output: Ordered list with correlations
    
    """
    
    print(f"Correlations of '{colname}':")
    
    correlations = []

    for i, j in dataframe.corr()[colname].iteritems():
        if abs(j) > min_corr:
            correlations.append((i, abs(j)))

    correlations.sort(key = lambda x: x[1], reverse=True)
    
    return correlations

for i in check_correlation(data, "site_eui", 0):
    print(i)

### Quick Findings

Based on above analysis, we try the following *as a starting point*.

> #### Delete the following columns
> - days_with_fog
> - direction_peak_wind_speed
> - direction_max_wind_speed
> - max_wind_speed
> - year_built
> - days_above_90F
> - days_above_110F
> - facility_type

> #### Impute missing values for energy_star_rating and year_built
> Imputing is done by taking the median

First tries are just a random forest model and xgboost.

#### Ideas

> - do PCA within years or something, construct new categories (like in titanic), then one hot encode
> - do something with average + st.dev of temperature or something! Write an algorythm that encapsulates this?
> - Keep in mind that huge differences in day and night are also importante
> - maybe take the diff (max-min) as a variable, normalize, multiply by normalized avg temperture per month.
> - sum al days above 80f and under 30f

In [ ]:
# Energy star rating
dat2 = data.loc[data["energy_star_rating"].isnull()]
del dat2["energy_star_rating"]

In [ ]:
distribution_plots(dat2)

In [ ]:
# Energy star rating
sub2 = sub.loc[sub["energy_star_rating"].isnull()]
del sub2["energy_star_rating"]

In [ ]:
distribution_plots(sub2)

In [ ]:
sub1 = sub.loc[sub["Year_Factor"] == 7]
del sub1["Year_Factor"]

In [ ]:
distribution_plots(sub1)

In [ ]:
sub3 = sub.loc[sub["State_Factor"] == "State_11"]
del sub3["State_Factor"]

# State 11 is only in year factor 7, so for state_11 all energy ratings are missing
# Small houses


In [ ]:
distribution_plots(sub3)

In [ ]:
sub4 = sub.loc[sub["Year_Factor"] == 7]
sub4 = sub4.loc[sub4["State_Factor"] == "State_11"]
distribution_plots(sub4)

In [ ]:
data1 = data.loc[data["State_Factor"] == "State_11"]
data1 = data1.loc[data1["State_Factor"] == "State_11"]
distribution_plots(data1)

In [ ]:
data1.floor_area

In [ ]:
print(data1.energy_star_rating.median())
print(data1.energy_star_rating.mode())
print(data1.energy_star_rating.mean())

In [ ]:
for i in check_correlation(data1, "site_eui", 0):
    print(i)

In [ ]:
facilities = test["facility_type"].unique()

In [ ]:
facilities_len = []
for i in facilities:
    facilities_len.append((i, len(test[test["facility_type"] == i])))

facilities_len.sort(key = lambda x: x[1], reverse=True)
facilities_len

In [ ]:
facilities_len.sort(key = lambda x: x[1], reverse=True)
facilities_len

In [ ]:
# Take intersection of state_11

facilities2 = set(facilities).intersection(set(sub4["facility_type"].unique()))

In [ ]:
facilities_len2 = []

for i in facilities2:
    facilities_len2.append((i, len(test[test["facility_type"] == i])))

facilities_len.sort(key = lambda x: x[1], reverse=True)
facilities_len

In [ ]:
# groupby facility type, take avg of energy star rating

t2 = test.groupby("facility_type").mean()

In [ ]:
# Education_College_or_university              52.076923
# doe maar 43.813559 voor food_service
# make a dictionary hieruit
# Warehouse nonrefrigirated 65.781910
# Laboratory 1.00
# Doe het met deze values!!! want t is allemaal in staat 11

t2["energy_star_rating"] 

In [ ]:
d32 = data.groupby("facility_type").mean()["energy_star_rating"]

In [ ]:
data.groupby("facility_type").count()